In [1]:
from MeetupHelperFunctions import *

import pandas as pd
import numpy as np

import os

import datetime as dt
import re
import gc
import time
import pickle

In [2]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import seaborn as sns
sns.set()

# To Plot matplotlib figures inline on the notebook
%matplotlib inline

sns.set(font_scale = 2)

#plt.style.use('fivethirtyeight') # pretty matplotlib plots
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (9, 5)
plt.rc('font', size=14)
plt.rc('figure', titlesize=18)
plt.rc('axes', labelsize=15)
plt.rc('axes', titlesize=18)

In [3]:
#folder containing csv files
data_folder = 'data/'

#folder containing Meetup groups data
group_folder = data_folder+'group_data_v2/'

#folder containing Meetup groups data
events_folder = data_folder+'event_data/'

#today's date
date_today = pd.to_datetime('today').floor(freq='D')

## Load and Clean the groups data

In [4]:
# get all category IDs
category_ids = list(pd.read_csv(data_folder+'categories.csv', usecols=['id'])['id'])

# merge all the groups csv files
df_list = []
for cat_id in category_ids:
    df = pd.read_csv(group_folder+'categoryID_'+str(cat_id)+'.csv', 
                     usecols=['id', 'name', 'category.id', 'category.name', 'category.shortname', 
                              'city', 'state', 'country', 'lat', 'lon', 'created', 
                              'members', 'description', 'organizer.member_id', 'rating', 
                              'topics', 'urlname', 'visibility', 'join_mode',
                              'last_event.id', 'last_event.name', 'last_event.time',
                              'last_event.utc_offset', 'last_event.yes_rsvp_count',
                              'next_event.id', 'next_event.name', 'next_event.time',
                              'next_event.utc_offset', 'next_event.yes_rsvp_count',
                              'membership_dues.required', 'membership_dues.trial_days',
                              'join_info.photo_req', 'join_info.questions_req',     
                              'who', 'sponsors', 'contributions.enabled'],
                     skipinitialspace=True)
    df['created'] = pd.to_datetime(df['created'], unit='ms').dt.floor(freq='D')
    df['last_event_time'] = df['last_event.time']+df['last_event.utc_offset']
    df['last_event_time'] = pd.to_datetime(df['last_event_time'], unit='ms').dt.floor(freq='D')
    df['next_event_time'] = df['next_event.time']+df['next_event.utc_offset']
    df['next_event_time'] = pd.to_datetime(df['next_event_time'], unit='ms').dt.floor(freq='D')
    
    df.drop(['last_event.time', 'last_event.utc_offset',
             'next_event.time', 'next_event.utc_offset'], axis=1, inplace=True)
    
    df_list.append(df)
    
groups_df = pd.concat(df_list, ignore_index=True)
groups_df.rename(columns={"id":"group_id", "name":"group_name",
                          "created":"group_created_date",
                          "organizer.member_id":"organizer_id",
                          "members":"group_member_count",
                          "rating":"group_rating"},
                 inplace=True)

# merge city and state fields
groups_df['group_location'] = (groups_df[['city', 'state']].copy()
                               .apply(lambda loc: '-'.join(loc) if loc[1] in us_states else loc[0], 
                                      axis=1)
                              )

# delete list not needed
del df_list[:] 
del df
gc.collect()

14

In [5]:
# get top locations based on number of groups
min_location_cnt = 10
min_mem_cnt = 5 
top_locations = list(groups_df.groupby(['group_location'])['group_id']
                     .agg('nunique')
                     .nlargest(min_location_cnt)
                     .index)

select_group_ids = list(groups_df[(groups_df['group_member_count']>min_mem_cnt) & 
                                  (groups_df['group_location'].isin(top_locations))]
                        ['group_id'].unique())
print('Number of Meetup groups selected: {}'.format(len(select_group_ids)))
print('Top {} locations:'.format(min_location_cnt))
top_locations

Number of Meetup groups selected: 36571
Top 10 locations:


['New York-NY',
 'San Francisco-CA',
 'Los Angeles-CA',
 'Chicago-IL',
 'San Diego-CA',
 'Austin-TX',
 'Denver-CO',
 'Seattle-WA',
 'Washington-DC',
 'Atlanta-GA']

In [6]:
print('Dimensions of full group dataframe:', groups_df.shape)
select_groups_df = groups_df[groups_df['group_id'].isin(select_group_ids)].copy()
print('Dimensions of selected group dataframe:', select_groups_df.shape)

print('Number of groups with missing organizer ids: ',sum(select_groups_df['organizer_id'].isnull()))
select_groups_df.dropna(subset=['group_id', 'organizer_id'], how='any', axis=0, inplace=True)
select_groups_df.drop_duplicates(subset=['group_id', 'organizer_id'], inplace=True)
print('Dimensions of selected group dataframe after cleaning:', select_groups_df.shape)

Dimensions of full group dataframe: (176312, 35)
Dimensions of selected group dataframe: (36571, 35)
Number of groups with missing organizer ids:  428
Dimensions of selected group dataframe after cleaning: (36143, 35)


In [7]:
# delete the bigger dataframe
del groups_df
gc.collect()

70

## Load and Clean organizers data

In [8]:
data_filenames = [f for f in os.listdir(data_folder) if os.path.isfile(data_folder+f)]
df_list = []
for file in data_filenames:
    if file.endswith(".csv"):
        if file.startswith('organizers_part'):
            print('Reading file: ', file)
            df = pd.read_csv(data_folder+file,
                             usecols=['id', 'city', 'state', 'country', 'lat', 'lon', 
                                      'membership_count', 'messaging_pref', 
                                      'name', 'topics', 'joined', 'visited'],
                             skipinitialspace=True,
                             low_memory=False)
            df['joined']  = pd.to_datetime(df['joined'],  unit='ms').dt.floor(freq='D')
            df['visited'] = pd.to_datetime(df['visited'], unit='ms').dt.floor(freq='D')
            df_list.append(df)
organizers_df = pd.concat(df_list, ignore_index=True)
organizers_df.rename(columns={"id"  : "organizer_id", 
                              "name": "organizer_name",
                              "membership_count":'num_active_membership',
                              "joined":"meetup_joined_date",
                              "visited":"most_recent_visit_date"},
                     inplace=True)

# delete list not needed
del df_list[:] 
del df
gc.collect()

Reading file:  organizers_part4.csv
Reading file:  organizers_part1.csv
Reading file:  organizers_part2.csv
Reading file:  organizers_part3.csv


42

In [9]:
print('Dimensions of organizers dataframe:', organizers_df.shape)
organizers_df.dropna(subset=['organizer_id'], axis=0, inplace=True)
organizers_df.drop_duplicates(subset=['organizer_id'], inplace=True)
print('Dimensions of organizers dataframe after cleaning:', organizers_df.shape)

Dimensions of organizers dataframe: (134105, 12)
Dimensions of organizers dataframe after cleaning: (123761, 12)


## load and clean Events data

In [10]:
## Merge the events csv files using the script ReadGroupEventsInfo2DF.py
# read the merged df
usecols = ['id', 'group.id', 'status', 'time', 'created', 'utc_offset', 'yes_rsvp_count', 'maybe_rsvp_count']
data_filenames = [f for f in os.listdir(data_folder) if os.path.isfile(data_folder+f)]
df_list = []
for file in data_filenames:
    if file.endswith(".csv"):        
        if file.startswith('all_group_events_script'):
            print('Reading file:', file)
            df = pd.read_csv(data_folder+file, 
                             usecols=usecols,
                             low_memory=False,
                             skipinitialspace=True)
            df_list.append(df)
            
all_events_df = pd.concat(df_list, ignore_index=True)
all_events_df.rename(columns={'group.id':'group_id',
                              'status':'event_status',
                              'id':'event_id'},
                    inplace=True)
del df_list[:]
gc.collect()

Reading file: all_group_events_script.csv
Reading file: all_group_events_script_part2.csv


74

In [11]:
event_status_sel = ['past', 'upcoming']
all_events_df = all_events_df[all_events_df['event_status'].isin(event_status_sel)].copy()

print('Dimensions of events dataframe:', all_events_df.shape)
all_events_df.dropna(subset=['group_id', 'event_id'], how='any', axis=0, inplace=True)
all_events_df.drop_duplicates(subset=['group_id', 'event_id'], inplace=True)
print('Dimensions of events dataframe after cleaning:', all_events_df.shape)

all_events_df['event_status'].value_counts()

Dimensions of events dataframe: (3448596, 8)
Dimensions of events dataframe after cleaning: (3423286, 8)


past        2977594
upcoming     445692
Name: event_status, dtype: int64

In [12]:
all_events_df['event_time'] = all_events_df['time']+all_events_df['utc_offset']
all_events_df['event_time'] = pd.to_datetime(all_events_df['event_time'], unit='ms').dt.floor(freq='D')
all_events_df.drop(['time','utc_offset'], axis=1, inplace=True)

In [13]:
def num_past_events(series):
    today2event_days = (date_today - series).dt.days
    past_events = today2event_days[today2event_days>=0]
    if len(past_events)==0:
        return 0
    else:
        return len(past_events)

def most_recent(series):
    today2event_days = (date_today - series).dt.days
    past_events = today2event_days[today2event_days>=0]
    if len(past_events)==0:
        future_events = np.abs(today2event_days[(today2event_days<0)&
                                                (np.abs(today2event_days)<31)]) # event within one month from today
        if len(future_events)==0:
            return None
        else:
            return min(future_events)
    else:
        return min(past_events)
    
def mean_freq(series):
    past = series[series<=date_today].copy()
    if not past.empty:
        return past.diff().mean().days
    else:
        future = series[series>date_today].copy()
        if not future.empty:
            return future.diff().mean().days
        else:
            return None

In [14]:
agg_func = {'yes_rsvp_count':['sum','mean'], 
            'maybe_rsvp_count':['sum','mean'],
            'event_time':['min', 'max', num_past_events, most_recent, mean_freq], 
            'event_status':['count']}
events_group_df = all_events_df.groupby('group_id').agg(agg_func).reset_index()

events_group_df.columns = ['_'.join(list(col)).strip('_') for col in events_group_df.columns.tolist()]

events_group_df['group_event1_age_days'] = ((date_today -
                                             events_group_df['event_time_min'])
                                             .dt.days)
events_group_df['group_eventN_age_days'] = ((date_today -
                                             events_group_df['event_time_max'])
                                             .dt.days)
events_group_df.rename(columns={'event_status_count':'all_event_count',
                                'event_time_most_recent':'event_most_recent',
                                'event_time_mean_freq':'events_mean_freq',
                                'event_time_num_past_events':'past_event_count'},
                       inplace=True)

# delete the bigger dataframe
del all_events_df
gc.collect()

print('Number of unique groups with event information:', events_group_df.shape[0])

Number of unique groups with event information: 28714


## Find subset of organizer and groups common to all dataframes and trim them

In [15]:
select_group_ids = list( 
                        set(select_groups_df['group_id'].unique()) & 
                        set(events_group_df['group_id'].unique())
                       )
print('Number of groups selected: {}'.format(len(select_group_ids)))

select_groups_df = select_groups_df[select_groups_df['group_id'].isin(select_group_ids)].copy()
print('Dimensions of groups dataframe after selection:', select_groups_df.shape)
events_group_df  = events_group_df[events_group_df['group_id'].isin(select_group_ids)].copy()
print('Dimensions of events dataframe after selection:', events_group_df.shape)

Number of groups selected: 18111
Dimensions of groups dataframe after selection: (18111, 35)
Dimensions of events dataframe after selection: (18111, 13)


In [16]:
select_organizer_ids = list( 
                            set(select_groups_df['organizer_id'].unique()) & 
                            set(organizers_df['organizer_id'].unique())
                           )
print('Number of organizers selected: {}'.format(len(select_organizer_ids)))

select_groups_df     = select_groups_df[select_groups_df['organizer_id'].isin(select_organizer_ids)].copy()
print('Dimensions of groups dataframe after selection:', select_groups_df.shape)
select_organizers_df = organizers_df[organizers_df['organizer_id'].isin(select_organizer_ids)].copy()
print('Dimensions of organizers dataframe after selection:', select_organizers_df.shape)

Number of organizers selected: 13850
Dimensions of groups dataframe after selection: (18111, 35)
Dimensions of organizers dataframe after selection: (13850, 12)


In [17]:
# delete the bigger dataframe
del organizers_df
gc.collect()

28

## Add/Analyze select group dataframe features

In [18]:
organizer_group_count_df = (select_groups_df.groupby(['organizer_id', 'group_id'])
                            .size()
                            .groupby('organizer_id')
                            .size())
organizers_with_more_than_1group_df = (organizer_group_count_df[organizer_group_count_df > 1]
                                       .reset_index()[['organizer_id']])
print('# Organizers with more than 1 group: %d' % len(organizers_with_more_than_1group_df))

# Organizers with more than 1 group: 2968


In [19]:
print('Group location stats:')
print(select_groups_df['group_location'].value_counts())

Group location stats:
New York-NY         6555
San Francisco-CA    3775
Los Angeles-CA      2999
Chicago-IL          2483
San Diego-CA        2299
Name: group_location, dtype: int64


In [20]:
print('Number of unique categories:', select_groups_df['category.id'].nunique())
print('Groups category stats:') 
select_groups_df['category.id'].describe()

Number of unique categories: 33
Groups category stats:


count    18111.000000
mean        19.162498
std         12.342277
min          1.000000
25%          6.000000
50%         21.000000
75%         32.000000
max         36.000000
Name: category.id, dtype: float64

In [21]:
#find age of group from today in months
select_groups_df['group_age_months'] = \
                        ((date_today.year - 
                          select_groups_df['group_created_date'].dt.year) * 12 +
                         (date_today.month - 
                          select_groups_df['group_created_date'].dt.month)
                        )
select_groups_df.loc[select_groups_df['group_age_months']==0, 'group_age_months'] = 0.5 
select_groups_df['log_group_age_months'] = (select_groups_df['group_age_months']
                                            .apply(lambda x : np.log10(x) if x>0 else 0))
select_groups_df['group_age_months'].describe()

count    18111.000000
mean        40.486445
std         38.407091
min          1.000000
25%         11.000000
50%         27.000000
75%         60.000000
max        186.000000
Name: group_age_months, dtype: float64

In [22]:
print('Percent of groups with rating 0 :{:.2f}%'.format(100*sum(select_groups_df['group_rating']==0)/
                                                       len(select_groups_df)))
print('Percent of groups with rating 5 :{:.2f}%'.format(100*sum(select_groups_df['group_rating']==5)/
                                                       len(select_groups_df)))
print('Group rating stats:')
select_groups_df['group_rating'].describe()

Percent of groups with rating 0 :33.10%
Percent of groups with rating 5 :21.13%
Group rating stats:


count    18111.000000
mean         3.170311
std          2.257647
min          0.000000
25%          0.000000
50%          4.670000
75%          4.940000
max          5.000000
Name: group_rating, dtype: float64

In [23]:
select_groups_df['log_group_mem_cnt'] = (select_groups_df['group_member_count']
                                          .apply(lambda x : np.log10(x) if x>0 else 0))

print('Group member count stats:')
select_groups_df['group_member_count'].describe()

Group member count stats:


count    18111.000000
mean       734.303793
std       1604.050585
min          6.000000
25%         79.000000
50%        242.000000
75%        721.000000
max      41861.000000
Name: group_member_count, dtype: float64

In [24]:
print('Group join mode stats:')
print(select_groups_df['join_mode'].value_counts())

Group join mode stats:
open        16361
approval     1744
closed          6
Name: join_mode, dtype: int64


In [25]:
print('Group visibility stats:')
select_groups_df['visibility'].value_counts()

Group visibility stats:


public            18109
public_limited        2
Name: visibility, dtype: int64

In [26]:
# check if group has sponsors
select_groups_df['has_sponsors'] = (select_groups_df['sponsors']
                                    .apply(lambda text: False if text=='[]' else True)
                                    .copy())
select_groups_df['has_sponsors'].value_counts()

False    13751
True      4360
Name: has_sponsors, dtype: int64

In [27]:
# extract the list of topic names and ids for each group
select_groups_df['group_topic_names'] = select_groups_df['topics'].copy().apply(parse_topics, args=('name',))
select_groups_df['group_topic_ids']   = select_groups_df['topics'].copy().apply(parse_topics, args=('id',))
select_groups_df['group_topic_cnt']   = select_groups_df['group_topic_ids'].apply(len)
select_groups_df.drop(['topics'], axis=1, inplace=True)

#check which groups have no topics
print('Number of groups with no topics:{}/{}'.format(
      sum(select_groups_df['group_topic_cnt']<1),len(select_groups_df)))
print('Percent of groups with no topics:{:.2f}%'.format(
      100*sum(select_groups_df['group_topic_cnt']<1)/len(select_groups_df)))

Number of groups with no topics:0/18111
Percent of groups with no topics:0.00%


In [28]:
#update the last event with next event date if next event is past today
def update_last_time(etimes):
    last_event, next_event = etimes
    if pd.isnull(next_event):
        return last_event
    elif (next_event>date_today):
        return last_event
    else:
        return next_event

select_groups_df['last_event_time_mod'] = (select_groups_df[['last_event_time','next_event_time']]
                                           .apply(update_last_time,
                                                  axis=1)
                                          )
#find days to last event from today
select_groups_df['group_last_event_days'] = ((date_today - select_groups_df['last_event_time_mod'])
                                             .dt.days)
select_groups_df['group_last_event_days'].describe()

count    17919.000000
mean       218.745019
std        364.151540
min          0.000000
25%         11.000000
50%         56.000000
75%        263.000000
max       4383.000000
Name: group_last_event_days, dtype: float64

In [29]:
events_group_df.describe()

,yes_rsvp_count_sum,yes_rsvp_count_mean,maybe_rsvp_count_sum,maybe_rsvp_count_mean,past_event_count,event_most_recent,events_mean_freq,all_event_count,group_event1_age_days,group_eventN_age_days
count,18111.000000,18111.000000,18111.000000,18111.000000,18111.000000,18066.000000,15674.000000,18111.000000,18111.000000,18111.000000
mean,1332.723649,16.159250,26.855116,0.111729,98.190105,223.879055,48.659117,111.816299,1172.978135,141.119817
std,5043.697608,27.647091,193.911367,0.632504,326.746129,381.129988,81.154382,409.589872,1151.019860,470.031969
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-321.000000,-14713.000000
25%,24.000000,2.889451,0.000000,0.000000,3.000000,9.000000,10.000000,4.000000,270.000000,-27.000000
50%,123.000000,6.972973,0.000000,0.000000,13.000000,54.000000,27.000000,17.000000,771.000000,41.000000
75%,671.000000,17.000000,0.000000,0.000000,64.000000,267.000000,54.000000,77.000000,1768.000000,260.000000
max,133246.000000,674.285714,7906.000000,19.712500,10024.000000,4383.000000,1693.000000,26960.000000,5770.000000,4383.000000


In [30]:
#Merge events with groups dataframe
select_groups_df = (select_groups_df.merge( events_group_df, how='left',
                                            left_on='group_id', right_on='group_id')
                   )
select_groups_df.drop(['event_time_min','event_time_max'], axis=1, inplace=True)

In [31]:
print('Selected Groups dataframe memory usage: {}'
      .format(mem_usage(select_groups_df)))

Selected Groups dataframe memory usage: 55.08 MB


In [32]:
category_cols = ['group_id', 'organizer_id', 'category.id', 'join_mode', 'visibility', 'group_location']
select_groups_df = change_dtype(select_groups_df, category_cols, verbose = False)

print('Selected Groups dataframe optimized memory usage: {}'
      .format(mem_usage(select_groups_df)))
select_groups_df.info()

Selected Groups dataframe optimized memory usage: 51.29 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18111 entries, 0 to 18110
Data columns (total 53 columns):
category.id                   18111 non-null category
category.name                 18111 non-null object
category.shortname            18111 non-null object
city                          18111 non-null object
contributions.enabled         17563 non-null object
country                       18111 non-null object
group_created_date            18111 non-null datetime64[ns]
description                   18092 non-null object
group_id                      18111 non-null category
join_info.photo_req           18111 non-null int64
join_info.questions_req       18111 non-null int64
join_mode                     18111 non-null category
last_event.id                 17743 non-null object
last_event.name               17743 non-null object
last_event.yes_rsvp_count     17743 non-null float32
lat                           18111 non

## Add/Analyze select organizer dataframe features

In [33]:
# merge city and state fields when country = us
def getOrganizerCountry(loc):
    if (loc is not np.nan):
        if loc.lower()=='us':
            return 'USA'
        else:
            return 'notUSA'
    else:
        return 'unknown'
    
select_organizers_df['organizer_country'] = (select_organizers_df['country'].copy()
                                               .apply(getOrganizerCountry))
print(select_organizers_df['organizer_country'].value_counts())

USA       13466
notUSA      384
Name: organizer_country, dtype: int64


In [34]:
# extract the list of topic names and ids for each organizer
select_organizers_df['organizer_topic_names'] = (select_organizers_df['topics']
                                                 .copy()
                                                 .apply(parse_topics, args=('name',)))
select_organizers_df['organizer_topic_ids']   = (select_organizers_df['topics']
                                                 .copy()
                                                 .apply(parse_topics, args=('id',)))
select_organizers_df['organizer_topic_cnt']   = (select_organizers_df['organizer_topic_ids']
                                                 .apply(len))

select_organizers_df.drop(['topics'], axis=1, inplace=True)

#check which organizers have no topics
print('Number of organizers with no topics:{}/{}'.format(
      sum(select_organizers_df['organizer_topic_cnt']<1),len(select_organizers_df)))
print('Percent of organizers with no topics:',
      100*sum(select_organizers_df['organizer_topic_cnt']<1)/len(select_organizers_df))

Number of organizers with no topics:3289/13850
Percent of organizers with no topics: 23.747292418772563


In [35]:
#find organizer membership age from today in months
select_organizers_df['membership_age_months'] = \
                        ((date_today.year - 
                          select_organizers_df['meetup_joined_date'].dt.year) * 12 +
                         (date_today.month - 
                          select_organizers_df['meetup_joined_date'].dt.month)
                        )
select_organizers_df.loc[select_organizers_df['membership_age_months']==0, 'membership_age_months'] = 0.5 

select_organizers_df['log_membership_age_months'] = (select_organizers_df['membership_age_months']
                                          .apply(lambda x : np.log10(x) if x>0 else 0))

select_organizers_df['membership_age_months'].describe()

count    13850.000000
mean        63.459567
std         42.401767
min          1.000000
25%         27.000000
50%         59.000000
75%         94.000000
max        190.000000
Name: membership_age_months, dtype: float64

In [36]:
#find organizer's last visit from today
select_organizers_df['last_visit_days'] = ((date_today -
                                           select_organizers_df['most_recent_visit_date'])
                                           .dt.days)
select_organizers_df['log_last_visit_days'] = (select_organizers_df['last_visit_days']
                                          .apply(lambda x : np.log10(x) if x>0 else 0))

select_organizers_df['last_visit_days'].describe()

count    13850.000000
mean        64.535884
std        142.760374
min          4.000000
25%         10.000000
50%         13.000000
75%         46.000000
max       2309.000000
Name: last_visit_days, dtype: float64

In [37]:
print("Organizer's messaging preference:")
select_organizers_df['messaging_pref'].value_counts()

Organizer's messaging preference:


all_members    11449
orgs_only       1580
groups_only      821
Name: messaging_pref, dtype: int64

In [38]:
select_organizers_df['num_active_membership'] = select_organizers_df['num_active_membership'].astype(int)
#q95 = np.percentile(select_organizers_df['num_active_membership'],95)

#select_organizers_df.loc[select_organizers_df['num_active_membership']>q95,'num_active_membership'] = q95+1
select_organizers_df['log_num_active_membership'] = (select_organizers_df['num_active_membership']
                                          .apply(lambda x : np.log10(x) if x>0 else 0))

print('Number of active memberships of organizers:')
select_organizers_df['num_active_membership'].describe()

Number of active memberships of organizers:


count    13850.000000
mean        15.424116
std         32.075099
min          0.000000
25%          1.000000
50%          6.000000
75%         18.000000
max       1094.000000
Name: num_active_membership, dtype: float64

In [39]:
# get the first and last group created by each organizer and number of groups
organizer_minmaxnum_group = (select_groups_df.groupby(['organizer_id'], sort=False)
                              ['group_created_date']
                              .agg([np.min, np.max])
                              .reset_index())
organizer_minmaxnum_group['organizer_grp1_age_days'] = ((date_today -
                                                       organizer_minmaxnum_group['amin'])
                                                       .dt.days)
organizer_minmaxnum_group['organizer_grpN_age_days'] = ((date_today -
                                                       organizer_minmaxnum_group['amax'])
                                                       .dt.days)
organizer_minmaxnum_group = (organizer_minmaxnum_group.merge((select_groups_df.groupby(['organizer_id'])
                                                                            ['group_id']
                                                                            .agg('nunique')
                                                                            .reset_index()),
                                                             how='left',
                                                             left_on='organizer_id', 
                                                             right_on='organizer_id'))
organizer_minmaxnum_group.rename(columns={'amin':'grp1_created_date',
                                          'amax':'grpN_created_date',
                                          'group_id':'num_groups'},
                                inplace=True)

organizer_minmaxnum_group.describe()

,organizer_grp1_age_days,organizer_grpN_age_days,num_groups
count,13850.000000,13850.000000,13850.000000
mean,1231.207798,1063.010614,1.307653
std,1195.610335,1067.458539,0.704221
min,13.000000,13.000000,1.000000
25%,294.000000,254.000000,1.000000
50%,811.000000,696.000000,1.000000
75%,1847.000000,1535.750000,1.000000
max,5654.000000,5654.000000,14.000000


In [40]:
select_organizers_df = (select_organizers_df.merge( organizer_minmaxnum_group, how='left',
                                                    left_on='organizer_id', right_on='organizer_id')
                       )
print('Dimension of merged organizer dataframe:',select_organizers_df.shape)

# delete dataframe after merging
del organizer_minmaxnum_group
gc.collect()

Dimension of merged organizer dataframe: (13850, 25)


400

In [41]:
select_organizers_df['log_grpN_age_days'] = (select_organizers_df['organizer_grpN_age_days']
                                          .apply(lambda x : np.log10(x) if x>0 else 0))

In [42]:
print('Selected Organizers dataframe memory usage: {}'
      .format(mem_usage(select_organizers_df)))

Selected Organizers dataframe memory usage: 12.66 MB


In [43]:
category_cols = ['organizer_id', 'messaging_pref', 'organizer_country']
select_organizers_df = change_dtype(select_organizers_df, category_cols, verbose = False)

print('Selected Organizers dataframe optimized memory usage: {}'
      .format(mem_usage(select_organizers_df)))

select_organizers_df.info()

Selected Organizers dataframe optimized memory usage: 11.02 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 13850 entries, 0 to 13849
Data columns (total 26 columns):
city                         13850 non-null object
country                      13850 non-null object
organizer_id                 13850 non-null category
meetup_joined_date           13850 non-null datetime64[ns]
lat                          13850 non-null object
lon                          13850 non-null float32
num_active_membership        13850 non-null int64
messaging_pref               13850 non-null category
organizer_name               13850 non-null object
state                        13587 non-null object
most_recent_visit_date       13850 non-null datetime64[ns]
organizer_country            13850 non-null category
organizer_topic_names        13850 non-null object
organizer_topic_ids          13850 non-null object
organizer_topic_cnt          13850 non-null int64
membership_age_months        13850 non-nul

## Save dataframes

In [44]:
select_groups_df.columns

Index(['category.id', 'category.name', 'category.shortname', 'city',
       'contributions.enabled', 'country', 'group_created_date', 'description',
       'group_id', 'join_info.photo_req', 'join_info.questions_req',
       'join_mode', 'last_event.id', 'last_event.name',
       'last_event.yes_rsvp_count', 'lat', 'lon', 'group_member_count',
       'membership_dues.required', 'membership_dues.trial_days', 'group_name',
       'next_event.id', 'next_event.name', 'next_event.yes_rsvp_count',
       'organizer_id', 'group_rating', 'sponsors', 'state', 'urlname',
       'visibility', 'who', 'last_event_time', 'next_event_time',
       'group_location', 'group_age_months', 'log_group_age_months',
       'log_group_mem_cnt', 'has_sponsors', 'group_topic_names',
       'group_topic_ids', 'group_topic_cnt', 'last_event_time_mod',
       'group_last_event_days', 'yes_rsvp_count_sum', 'yes_rsvp_count_mean',
       'maybe_rsvp_count_sum', 'maybe_rsvp_count_mean', 'past_event_count',
       'eve

In [45]:
group_select_cols = ['category.id', 'category.name',
                     'group_id', 'group_name',
                     'organizer_id', 
                     'join_mode', 
                     'group_member_count', 'log_group_mem_cnt',
                     'group_rating', 
                     'visibility', 
                     'group_location', 
                     'group_age_months', 'log_group_age_months',
                     'has_sponsors',
                     'group_topic_names', 'group_topic_ids', 'group_topic_cnt',
                     'group_last_event_days',
                     'yes_rsvp_count_mean', 'maybe_rsvp_count_mean',
                     'event_most_recent', 'events_mean_freq', 
                     'past_event_count', 'all_event_count',
                     'group_event1_age_days', 'group_eventN_age_days']

In [55]:
# Check if any columns have null values
null_check_df = check_nulls(select_groups_df, group_select_cols, verbose = False)
print('Columns with nulls:',
      null_check_df.columns[~null_check_df.isin([0]).any()].tolist())
null_check_df

Columns with nulls: ['group_last_event_days', 'event_most_recent', 'events_mean_freq']


,category.id,category.name,group_id,group_name,organizer_id,join_mode,group_member_count,log_group_mem_cnt,group_rating,visibility,...,group_topic_cnt,group_last_event_days,yes_rsvp_count_mean,maybe_rsvp_count_mean,event_most_recent,events_mean_freq,past_event_count,all_event_count,group_event1_age_days,group_eventN_age_days
column_type,category,object,category,object,category,category,int64,float32,float32,category,...,int64,float32,float32,float32,float32,float32,int64,int64,int64,int64
null_count,0,0,0,0,0,0,0,0,0,0,...,0,192,0,0,45,2437,0,0,0,0
%_null_count,0,0,0,0,0,0,0,0,0,0,...,0,1.06013,0,0,0.248468,13.4559,0,0,0,0


In [47]:
select_groups_df[group_select_cols].shape

(18111, 26)

In [48]:
select_groups_df[group_select_cols].head(2)

,category.id,category.name,group_id,group_name,organizer_id,join_mode,group_member_count,log_group_mem_cnt,group_rating,visibility,...,group_topic_cnt,group_last_event_days,yes_rsvp_count_mean,maybe_rsvp_count_mean,event_most_recent,events_mean_freq,past_event_count,all_event_count,group_event1_age_days,group_eventN_age_days
0,1,fine arts/culture,2657,The LA Acting Meetup Group Sponsor PerformerTr...,294838.0,open,490,2.690196,4.81,public,...,5,355.0,3.212963,0.361111,355.0,21.0,216,216,4939,355
1,1,fine arts/culture,153363,Hollywood- LA Theatre Meetup Sponsored by Perf...,294838.0,open,323,2.509202,4.93,public,...,6,355.0,1.432558,0.083721,355.0,21.0,215,215,4938,355


In [49]:
select_groups_df.to_csv(data_folder+'groups_select_info_v2.csv', 
                        columns=group_select_cols, 
                        index=False)

with open(data_folder+'groups_select_info_v2.pkl', "wb") as fp:  
    pickle.dump(select_groups_df[group_select_cols], fp)

In [50]:
select_organizers_df.columns

Index(['city', 'country', 'organizer_id', 'meetup_joined_date', 'lat', 'lon',
       'num_active_membership', 'messaging_pref', 'organizer_name', 'state',
       'most_recent_visit_date', 'organizer_country', 'organizer_topic_names',
       'organizer_topic_ids', 'organizer_topic_cnt', 'membership_age_months',
       'log_membership_age_months', 'last_visit_days', 'log_last_visit_days',
       'log_num_active_membership', 'grp1_created_date', 'grpN_created_date',
       'organizer_grp1_age_days', 'organizer_grpN_age_days', 'num_groups',
       'log_grpN_age_days'],
      dtype='object')

In [56]:
organizer_select_cols = ['organizer_id', 
                         'num_active_membership', 'log_num_active_membership',
                         'messaging_pref',
                         'organizer_country', 
                         'organizer_topic_names', 'organizer_topic_ids', 'organizer_topic_cnt', 
                         'membership_age_months', 'log_membership_age_months',
                         'last_visit_days', 'log_last_visit_days',
                         'organizer_grpN_age_days', 'log_grpN_age_days', 
                         'num_groups']

In [57]:
# Check if any columns have null values
null_check_df = check_nulls(select_organizers_df, organizer_select_cols, verbose = False)
print('Columns with nulls:',
      null_check_df.columns[~null_check_df.isin([0]).any()].tolist())
null_check_df

Columns with nulls: []


,organizer_id,num_active_membership,log_num_active_membership,messaging_pref,organizer_country,organizer_topic_names,organizer_topic_ids,organizer_topic_cnt,membership_age_months,log_membership_age_months,last_visit_days,log_last_visit_days,organizer_grpN_age_days,log_grpN_age_days,num_groups
column_type,category,int64,float32,category,category,object,object,int64,float32,float32,int64,float32,int64,float32,int64
null_count,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
%_null_count,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [58]:
select_organizers_df[organizer_select_cols].shape

(13850, 15)

In [59]:
select_organizers_df[organizer_select_cols].head(2)

,organizer_id,num_active_membership,log_num_active_membership,messaging_pref,organizer_country,organizer_topic_names,organizer_topic_ids,organizer_topic_cnt,membership_age_months,log_membership_age_months,last_visit_days,log_last_visit_days,organizer_grpN_age_days,log_grpN_age_days,num_groups
0,246253340.0,2,0.301030,all_members,USA,[],[],0,3.0,0.477121,16,1.20412,29,1.462398,1
1,6721449.0,86,1.934498,all_members,notUSA,"[Drama, Book Club, Women's Book Club]","[21022, 2767, 26397]",3,121.0,2.082785,5,0.69897,184,2.264818,5


In [60]:
select_organizers_df.to_csv(data_folder+'organizers_select_info_v2.csv', 
                            columns=organizer_select_cols, 
                            index=False)
                         
with open(data_folder+'organizers_select_info_v2.pkl', "wb") as fp:  
    pickle.dump(select_organizers_df[organizer_select_cols], fp)